# Create OCEAN Data NFT
Quickly create OCEAN Data NFTs by querying blockchain data from Dune and Flipside.

**Steps:**
1. Get Data
2. Store Data
3. Create Data NFT

Alternative Option: Build Dune/Flipside queries on platforms and create OCEAN Data NFTs from UI.

1. Get "out of gas" error a lot
2. How to add key/headers in publish for url 
3. Missing pricing schema
4. Confirm can download file 

In [1]:
import json
import pandas as pd
import os
from dotenv import load_dotenv
import requests
import subprocess
from flipside import Flipside
from ocean_lib.web3_internal.utils import connect_to_network
from ocean_lib.example_config import get_config_dict
from ocean_lib.ocean.ocean import Ocean
from brownie.network import accounts

# Load variables from .env file
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")
FLIPSIDE_API_KEY = os.getenv("FLIPSIDE_API_KEY")
GITHUB_KEY = os.getenv("GITHUB_KEY")

## 1. Get Data
### Flipside

* SDK Docs: https://docs.flipsidecrypto.com/flipside-api/get-started

In [2]:
# Query Flipside using their Python SDK
def query_flipside(sql):
    flipside = Flipside(FLIPSIDE_API_KEY, "https://api-v2.flipsidecrypto.xyz")
    results = flipside.query(sql)
    results_df = pd.DataFrame(results.rows, columns=results.columns).drop(columns=["__row_index"])
    return results_df

In [3]:
sql = """
SELECT
  date(block_timestamp) as dt,
  count(distinct tx_hash) as tx_ct
FROM ethereum.core.fact_transactions
WHERE block_timestamp >= GETDATE() - interval'7 days'
GROUP BY 1
order by 1 asc
"""
results_df = query_flipside(sql)
results_df

,dt,tx_ct
0,2023-08-08T00:00:00.000Z,115922
1,2023-08-09T00:00:00.000Z,1076595
2,2023-08-10T00:00:00.000Z,1037733
3,2023-08-11T00:00:00.000Z,1050203
4,2023-08-12T00:00:00.000Z,972839
5,2023-08-13T00:00:00.000Z,920802
6,2023-08-14T00:00:00.000Z,1009971
7,2023-08-15T00:00:00.000Z,948150


### Dune

* API Docs: https://dune.com/docs/api/

In [4]:
# Query Dune Analytics via API
def query_dune(q):
    url = f"https://api.dune.com/api/v1/query/{q}/results?api_key={DUNE_API_KEY}"
    response = requests.get(url)
    results_json = json.loads(response.text)["result"]["rows"]
    results_df = pd.DataFrame.from_dict(results_json)
    return results_df

In [5]:
dune_query_id = 2847682
results_df = query_dune(dune_query_id)
results_df

,dt,tx_ct
0,2023-08-05,1124372
1,2023-08-06,987506
2,2023-08-07,1047336
3,2023-08-08,1098689
4,2023-08-09,1076595
5,2023-08-10,1037733
6,2023-08-11,1050203
7,2023-08-12,98194


## 2. Store Data
### Add File to GitHub

*Note: This is a quick & easy solution to host the data, but can also add file to IPFS, Arweave, GCP, AWS, etc...*

In [6]:
# Create File from Query results
results_df.to_csv('query_results3.csv', index=False)

In [26]:
# Store file in GitHub repo
def git_add_commit_push(commit_message):
    try:
        subprocess.run(['git', 'add', '.'])
        subprocess.run(['git', 'commit', '-m', commit_message])
        subprocess.run(['git', 'push'])
        print("Git add, commit, and push successful!")
    except Exception as e:
        print("An error occurred:", e)

# Replace 'Your commit message here' with your desired commit message
commit_message = 'Your commit message here'

git_add_commit_push(commit_message)

[main 354fef2] Your commit message here
 2 files changed, 85 insertions(+), 21 deletions(-)
 delete mode 100644 query_results.csv
Git add, commit, and push successful!


To https://github.com/PrimoData/ocean_data_nfts
   22d974c..354fef2  main -> main


## 3. Create Data NFT
### Add Data NFT to the OCEAN Protocol Marketplace

* Marketplace: https://market.oceanprotocol.com/
* SDK Docs: https://docs.oceanprotocol.com/developers/ocean.py/install

In [10]:
# Connect to Blockchain
connect_to_network("polygon-main")
config = get_config_dict("polygon-main")
ocean = Ocean(config)

In [21]:
# Connect to Wallet
accounts.clear()
private_key = os.getenv('PRIVATE_KEY')
wallet = accounts.add(private_key)
print('MATIC Balance:', wallet.balance() / 1e18 )

MATIC Balance: 5.586549830042127


In [24]:
# Create Data NFT
name = "Data NFT Test 3"

# Public GitHub
url = "https://raw.githubusercontent.com/PrimoData/ocean_data_nfts/main/query_results2.csv"

# Private GitHub
#url = f"https://raw.githubusercontent.com/PrimoData/ocean_data_nfts/main/query_results2.csv?token={GITHUB_KEY}"

tx_dict = {
    "from": wallet,
    "priority_fee": "500 gwei",
    "max_fee": "500 gwei",
}
(data_nft, datatoken, ddo) = ocean.assets.create_url_asset(name, url, tx_dict)

print("Data NFT published:")
print(f"  data_nft: symbol={data_nft.symbol()}, address={data_nft.address}")
print(f"  datatoken: symbol={datatoken.symbol()}, address={datatoken.address}")
print(f"  did={ddo.did}")

Transaction sent: 0x08e38fce1abf314ff5917c1935feee5652b2ddd09c742cc0ed923e3dbd4809bd
  Max fee: 500.0 gwei   Priority fee: 500.0 gwei   Gas limit: 1811594   Nonce: 32
  ERC721Factory.createNftWithErc20 confirmed   Block: 46299425   Gas used: 1595610 (88.08%)   Gas price: 500.0 gwei



INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x044f3de2f72d999b4b921f180ccc3d7f79e2cae001622c53eca8ece858c42116a2f8f291368aa2be004728420970a3492edb61d5a53cec7b39dfa8d8daac53a4a3e15aa3fa2798111f898ff609ce4feadfc834901766beab91022e46551a26481bfa47f4292480b80483657337c90203dd7d793025e63ad88a9e0bd8fc146d299900ecda7d22f93a8b1e3dcc04c41ab90b4aedd3baf6d61efb937d5f918c2ee54ce277368358e529690e415c6b2c20114a35bc00b66ca5919fce5c80ec6f29e81c32a4ffbf5921ea39f8c01af113a27462698e60a03faf9bd4da0910e0c7de4b5a10f3b398908917353d8060343e643964a533716bf332eb812a4bcf47eb5ffa4887a29e83b43ce83d563fda791b503ce2fcf0724a9b7f363af9f1a7079f5cc3d97888556ddc13185748fe5fea8fc2b50d36e94a9179f08577e6b20478eb2a973076aa43d6605ac620009d955d8db2e92f, encryptedEndpoint https://v4.provider.polygon.oceanprotocol.com/api/services/encrypt?chainId=137
INFO:ocean_lib.data_provider.data_encryptor:Asset urls encrypted successfully, encrypted urls str: 0x04afa0ef77cfab6ebbf48d968

ValueError: Execution reverted during call: 'out of gas'. This transaction will likely revert. If you wish to broadcast, include `allow_revert:True` as a transaction parameter.